In [1]:
import rdflib
from rdflib.namespace import RDF, RDFS, SH, XSD
from nanopub import (
    definitions,
    Nanopub,
    NanopubClient,
    load_profile,
    NanopubConf,
)
from nanopub.fdo import (
    FdoNanopub,
    FdoRecord,
    update_record,
    retrieve_record_from_id,
    retrieve_content_from_id,
    validate_fdo_record,
    resolve_id
)
import datetime
from nanopub.namespaces import HDL, FDOF, PROV, NPX, NP
from pyshacl import validate
from rdflib import Graph, URIRef, Literal
import requests
import json


In [2]:
## Setup

conf = NanopubConf(
    add_prov_generated_time=False,
    add_pubinfo_generated_time=True,
    attribute_assertion_to_profile=True,
    attribute_publication_to_profile=True,
    profile=load_profile(),
    use_test_server=True
)

## Define identifiers
iri = "https://w3id.org/np/RAsSeIyT03LnZt3QvtwUqIHSCJHWW1YeLkyu66Lg4FeBk/nanodash-readme"
handle_iri = "https://hdl.handle.net/21.T11966/82045bd97a0acce88378"
handle_pid = "21.T11966/82045bd97a0acce88378"


## Op.Resolve

Retrieving key-value pairs from the FDO record for given FDO PID.

In [3]:
fdo_record_from_iri = resolve_id(iri)
fdo_record_from_handle_iri = resolve_id(handle_iri)
fdo_record_from_handle_pid = resolve_id(handle_pid)

print(fdo_record_from_iri)
print(fdo_record_from_handle_iri)
print(fdo_record_from_handle_pid)

FDO Record
  ID: test-fdo-profile
  Label: Test FDO: README of Nanodash
  Profile: https://w3id.org/np/RABPR2eJ7dbuf_OPDLztvRZI-el2_wBFkVBiPCLmr1Q50/test-fdo-profile
FDO Record
  ID: 996c38676da9ee56f8ab
  Label: 21.T11966/82045bd97a0acce88378
  Profile: https://hdl.handle.net/21.T11966/996c38676da9ee56f8ab
FDO Record
  ID: 996c38676da9ee56f8ab
  Label: 21.T11966/82045bd97a0acce88378
  Profile: https://hdl.handle.net/21.T11966/996c38676da9ee56f8ab


## Op.RetrieveContent
Download the bitstream/file of an FDO.

In [4]:
## Utility function to print the results of this operation
def print_file_info(file_content: bytes, content_type: str = "unknown", filename: str = None):
    print(f"Filename: {filename or 'N/A'}")
    print(f"Content-Type: {content_type}")
    print(f"Size: {len(file_content)} bytes")
    print("Content preview (first 500 bytes):")
    try:
        print(file_content[:500].decode("utf-8"))
    except UnicodeDecodeError:
        print(file_content[:500])  # print raw bytes if not utf-8

fdo_content_from_iri = retrieve_content_from_id(iri)
print_file_info(fdo_content_from_iri)

# TODO: replace with an FDO example with valid dataref
#fdo_content_from_handle_iri = retrieve_content_from_id(handle_iri)
#print_file_info(fdo_content_from_handle_iri)

# TODO: replace with an FDO example with valid dataref
#fdo_content_from_handle_pid = retrieve_content_from_id(handle_pid)
#print_file_info(fdo_content_from_handle_pid)

Filename: N/A
Content-Type: unknown
Size: 2419 bytes
Content preview (first 500 bytes):
Nanodash

![logo](nanodash.png)

Nanodash was previously called Nanobench.

Nanodash is a client to browse and publish nanopublications.


### Online Instances

You can use Nanodash by login in via ORCID in one of the online instances:

- https://nanodash.petapico.org/
- https://nanodash.knowledgepixels.com/
- https://nanodash.np.trustyuri.net/


### Local Installation

To use Nanodash locally, see the [installation instructions with Docker](INSTALL-with-Docker.md).


### Screenshot

Th


## Op.Validate
Validate given FDO record according to its profile.

In [7]:
# Step 1: Resolve PID
fdo_record_from_iri = resolve_id(iri)

# Step 2: Validate
is_valid = validate_fdo_record(fdo_record_from_iri)
print("Valid FDO" if is_valid else "Invalid FDO")

# TODO: examples with valid handle-based FDOs
# Step 1: Resolve PID
#fdo_record_from_handle_iri = resolve_id(handle_iri)

# Step 2: Validate
#validate_fdo_record(fdo_record_from_handle_iri)

# Step 1: Resolve PID
#fdo_record_from_handle_pid = resolve_id(handle_pid)

# Step 2: Validate
#validate_fdo_record(fdo_record_from_handle_pid)


Validation failed:
 True
Valid FDO


## Op.Create
Defining a new FDO by creating and publishing a new FDO record (for an existing file, or without file reference).

In [6]:
record = FdoRecord(
    profile_uri="https://hdl.handle.net/12345",
    label="Test FDO",
    dataref="https://example.org/data/456"
)

fdo_from_iri = FdoNanopub.create_with_fdo_iri(fdo_record=record, fdo_iri="https://example.org/example-fdo")
print(fdo_from_iri)

@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix np: <http://www.nanopub.org/nschema#> .
@prefix npx: <http://purl.org/nanopub/x/> .
@prefix ns1: <https://w3id.org/fdof/ontology#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

<http://purl.org/nanopub/temp/np/Head> {
    <http://purl.org/nanopub/temp/np/> a np:Nanopublication ;
        np:hasAssertion <http://purl.org/nanopub/temp/np/assertion> ;
        np:hasProvenance <http://purl.org/nanopub/temp/np/provenance> ;
        np:hasPublicationInfo <http://purl.org/nanopub/temp/np/pubinfo> .
}

<http://purl.org/nanopub/temp/np/pubinfo> {
    <http://purl.org/nanopub/temp/np/> rdfs:label "FAIR Digital Object: Test FDO" ;
        npx:introduces <https://example.org/example-fdo> .
}

<http://purl.org/nanopub/temp/np/assertion> {
    <https://example.org/example-fdo> a ns1:FAIRDigitalObject ;
        rdfs:label "Test FDO" ;
        dcterms:conformsTo <https://hdl.handle.net/12345> ;
        ns1:hasMetadata <http://purl.org/

## Op.Handle2Nanopub

This operation is transforming an existing Handle-based FDO record into a nanopublication-based one. 

In [9]:
np = FdoNanopub.handle_to_nanopub(handle_pid)

print(np)

@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix np: <http://www.nanopub.org/nschema#> .
@prefix npx: <http://purl.org/nanopub/x/> .
@prefix ns1: <https://hdl.handle.net/10320/> .
@prefix ns2: <https://hdl.handle.net/21.T11966/> .
@prefix ns3: <https://hdl.handle.net/0.TYPE/> .
@prefix ns4: <https://w3id.org/fdof/ontology#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

<http://purl.org/nanopub/temp/np/Head> {
    <http://purl.org/nanopub/temp/np/> a np:Nanopublication ;
        np:hasAssertion <http://purl.org/nanopub/temp/np/assertion> ;
        np:hasProvenance <http://purl.org/nanopub/temp/np/provenance> ;
        np:hasPublicationInfo <http://purl.org/nanopub/temp/np/pubinfo> .
}

<http://purl.org/nanopub/temp/np/pubinfo> {
    <http://purl.org/nanopub/temp/np/> rdfs:label "FAIR Digital Object: 21.T11966/82045bd97a0acce88378" ;
        npx:introduces ns2:82045bd97a0acce88378 .
}

<http://purl.org/nanopub/temp/np/assertion> {
    ns2:82045bd97a0acce88378 a ns4:FAI

## Op.Update record

Updates the record of an FDO, keeping the same FDO PID.

In [3]:
# Step 1: Resolve PID (handle or IRI)
record = resolve_id(iri)
print(f"Resolved record:\n{record}\n")

# Step 2: Create a new FdoNanopub from the record 

npub = FdoNanopub.create_with_fdo_iri(
    fdo_record=record,
    fdo_iri="https://example.org/test-fdo",
    data_ref=record.get_data_ref(),
    conf=conf
)

npub.publish()

print(f"Created FdoNanopub with FDO IRI: {npub.source_uri}\n")

# Step 3: Update nanopub assertion from updated record 

record.set_property(RDFS.label, "Updated Label via RDFS.label")
np_uri = update_record(npub, record)
print(f"Updated nanopub with new record data. Nanopub URI: {np_uri}\n")

# Step 4: Sign the nanopub

npub.sign()
print("Nanopub signed.\n")

# Step 5: Publish the nanopub 

published_uri = npub.publish()
print(f"Nanopub published at URI: {published_uri}\n")


Resolved record:
FDO Record
  ID: test-fdo-profile
  Label: Test FDO: README of Nanodash
  Profile: https://w3id.org/np/RABPR2eJ7dbuf_OPDLztvRZI-el2_wBFkVBiPCLmr1Q50/test-fdo-profile

Created FdoNanopub with FDO IRI: https://w3id.org/np/RAAkEd5JKhtPFotAy6HADnaT3QXAhGxLPX6bJBNSrkka4

Updated nanopub with new record data. Nanopub URI: https://w3id.org/np/RAAkEd5JKhtPFotAy6HADnaT3QXAhGxLPX6bJBNSrkka4

Nanopub signed.

Nanopub published at URI: ('https://w3id.org/np/RATHZzW6eb-NpJT66seBWWUdInupY1YJv8UvZPuDhOkek', 'https://test.registry.knowledgepixels.com/np/')

